# NLP Machine Learning Project 2022

Useful links:

https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b
https://www.andyfitzgeraldconsulting.com/writing/keyword-extraction-nlp/

In [1]:
# pip install nltk

In [92]:
import nltk
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
# from term_frequency import term_frequencies, feature_names, df_term_frequencies

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
transformer = TfidfTransformer()
tt = TweetTokenizer()

In [93]:
# !pip install wordcloud

In [94]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [106]:
df = pd.read_csv(r'train.csv')

df.replace('NaN', np.NaN, inplace = True)

# to count the number of NaN's in each column, just change the column name in this line to see how many missing values of that
# variable per other column
# print(df[df.keyword.isnull()].count())

df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [107]:
def preprocess_tweets(text):
    
# remove mentions and URLs
    text_noMentionURL = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
    text_noMentionURL = " ".join(text_noMentionURL.split())
    
# remove '#' symbols and add space before capital letter
    text_noHash = re.sub(r"([A-Z]+)", r" \1", text)
    text_noHash = re.sub(r"(#)", "", text_noHash)
    text_noHash = " ".join(text_noHash.split())
    
# remove all other punctuation
    text_noNoise = "".join([char for char in text_noHash if char not in string.punctuation])

    return text_noNoise.lower()


df['tweet_noNoise'] = df["text"].apply(lambda x: preprocess_tweets(x))
df.head()

,id,keyword,location,text,target,tweet_noNoise
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [109]:
# lemmatizer with Part-of-Speech (POS)

from nltk.corpus import wordnet
from collections import Counter

# this function can be called within lemmatize() to tag the word with its POS
def get_pos(word):
    probable_part_of_speech = wordnet.synsets(word)
  
    pos_counts = Counter()

    pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
  
    most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
    return most_likely_part_of_speech

# tokenize tweets = divide into words
tokenized_tweets = df.apply(lambda row: tt.tokenize(row['tweet_noNoise']), axis=1)

# lemmatize the list of all tokens
df['lemmatized_list'] = tokenized_tweets.apply(lambda y: [lemmatizer.lemmatize(x, get_pos(x)) for x in y])
df['lemmatized_list'].apply(lambda x: ' '.join([tweet for tweet in x]))

# removes stop words (are, on, in, etc.) MUST COME BEFORE LIST -> STR
stop = set(stopwords.words('english'))
df['lemmatized_list'] = df['lemmatized_list'].apply(lambda x: [y for y in x if y not in stop])

# turn the resulting list back into a string
df['processed_tweet'] = df.lemmatized_list.agg(lambda x: ','.join(map(str, x)))


df.head()

,id,keyword,location,text,target,tweet_noNoise,lemmatized_list,processed_tweet
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[deed, reason, earthquake, may, allah, forgive...","deed,reason,earthquake,may,allah,forgive,u"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]","forest,fire,near,la,ronge,sask,canada"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[resident, ask, shelter, place, notify, office...","resident,ask,shelter,place,notify,officer,evac..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfire, evacuation,...","13000,people,receive,wildfire,evacuation,order..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[get, send, photo, ruby, alaska, smoke, wildfi...","get,send,photo,ruby,alaska,smoke,wildfire,pour..."


In [110]:
# enter whatever word you'd like to see how lemmatizing works
print(lemmatizer.lemmatize('deeds'))

deed


In [112]:
# first attempts at tf-idf
# input needs to exclude mentions & hashtags, be in lower case, remove punct, lemmatized

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(norm=None)
tfidf_scores = vectorizer.fit_transform(df.processed_tweet)

feature_names = vectorizer.get_feature_names()

# get corpus index
corpus_index = [f"Tweet {i+1}" for i in range(len(df.processed_tweet))]

# create pandas DataFrame with tf-idf scores
# try:
df_tf_idf = pd.DataFrame(tfidf_scores.T.todense(), index=feature_names, columns=corpus_index)
print(df_tf_idf)
# except:
    # pass

        Tweet 1  Tweet 2  Tweet 3  Tweet 4  Tweet 5  Tweet 6  Tweet 7  \
0000        0.0      0.0      0.0      0.0      0.0      0.0      0.0   
001         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
0011        0.0      0.0      0.0      0.0      0.0      0.0      0.0   
001116      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
0025        0.0      0.0      0.0      0.0      0.0      0.0      0.0   
...         ...      ...      ...      ...      ...      ...      ...   
ûó          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
ûóher       0.0      0.0      0.0      0.0      0.0      0.0      0.0   
ûókody      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
ûótech      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
ûówe        0.0      0.0      0.0      0.0      0.0      0.0      0.0   

        Tweet 8  Tweet 9  Tweet 10  ...  Tweet 7604  Tweet 7605  Tweet 7606  \
0000        0.0      0.0       0.0  ...     

In [66]:
# IGNORE FOR RIGHT NOW
# don't think we need to remove stop words with tf-idf but keep here

stop = set(stopwords.words('english'))

# removes 'stop words' such as 'the', 'are', etc. it knows these stop words where i defined 'stop' variable, comes from a library
df['tweet_stop'] = df['lemmatized_tweet'].apply(lambda x: [y for y in x if y not in stop])

# hashtags
df['hashtag'] = df.text.apply(lambda x: re.findall(r"#(\w+)", x))

df.head()

In [ ]:
# IGNORE FOR RIGHT NOW

df.tweet_stop = df.tweet_stop.apply(lambda x: [' '.join(str(y)) for y in x])
print(df.tweet_stop)

In [ ]:
# IGNORE FOR RIGHT NOW

# this is my attempt at creating a matrix of term frequencies using tf-idf but the problem is that
# a lot of the 'words' people use are literally jibberish

from sklearn.feature_extraction.text import CountVectorizer

df.tweet_stop = df.tweet_stop.apply(lambda x: [''.join(str(y)) for y in x])
# initialize and fit CountVectorizer
vectorizer = CountVectorizer()
term_frequencies = vectorizer.fit_transform(df.text)

# get vocabulary of terms
feature_names = vectorizer.get_feature_names()

# get corpus index
corpus_index = [f"Tweet {i+1}" for i in range(len(df.tokenized_tweet))]

# create pandas DataFrame with term frequencies
df_term_frequencies = pd.DataFrame(term_frequencies.T.todense(), index=feature_names, columns=corpus_index)

# df_term_frequencies.head(30)
print(df_term_frequencies.iloc[:])

df_term_frequencies['frequency_summation'] = df_term_frequencies.iloc[:].sum(axis=1)
print(df_term_frequencies.iloc[:])

df_term_frequencies = df_term_frequencies[df_term_frequencies['frequency_summation'] >= 2]
# df_term_frequencies = df_term_frequencies.loc[df_term_frequencies.frequency_summation >= 5]
print(df_term_frequencies.iloc[:])